In [1]:
import pandas as pd
import numpy as np

## 分组计算

分组计算三步曲：拆分 -> 应用 -> 合并

* 拆分：根据什么进行分组？
* 应用：每个分组进行什么样的计算？
* 合并：把每个分组的计算结果合并起来。


![groupby](imgs/groupby.png)

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randint(1, 10, 5),
                  'data2': np.random.randint(1, 10, 5)})
df

,data1,data2,key1,key2
0,1,6,a,one
1,5,9,a,two
2,4,7,b,one
3,3,7,b,two
4,3,5,a,one


### 对 Series 进行分组

通过索引对齐关联起来

In [3]:
grouped = df['data1'].groupby(df['key1'])

In [4]:
grouped.mean()

key1
a    3.0
b    3.5
Name: data1, dtype: float64

In [5]:
df['data1'].groupby([df['key1'], df['key2']]).mean()

key1  key2
a     one     2
      two     5
b     one     4
      two     3
Name: data1, dtype: int32

### 对 DataFrame 进行分组

In [6]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,3.0,6.666667
b,3.5,7.000000


In [7]:
means = df.groupby(['key1', 'key2']).mean()['data1']
means

key1  key2
a     one     2
      two     5
b     one     4
      two     3
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,2,5
b,4,3


In [9]:
df.groupby(['key1', 'key2'])['data1'].mean()

key1  key2
a     one     2
      two     5
b     one     4
      two     3
Name: data1, dtype: int32

### 每个分组的元素个数

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代

In [11]:
for name, group in df.groupby('key1'):
    print name
    print group


a
   data1  data2 key1 key2
0      1      6    a  one
1      5      9    a  two
4      3      5    a  one
b
   data1  data2 key1 key2
2      4      7    b  one
3      3      7    b  two


In [12]:
for name, group in df.groupby(['key1', 'key2']):
    print name
    print group

('a', 'one')
   data1  data2 key1 key2
0      1      6    a  one
4      3      5    a  one
('a', 'two')
   data1  data2 key1 key2
1      5      9    a  two
('b', 'one')
   data1  data2 key1 key2
2      4      7    b  one
('b', 'two')
   data1  data2 key1 key2
3      3      7    b  two


### 转化为字典

In [13]:
d = dict(list(df.groupby('key1')))
d

{'a':    data1  data2 key1 key2
 0      1      6    a  one
 1      5      9    a  two
 4      3      5    a  one, 'b':    data1  data2 key1 key2
 2      4      7    b  one
 3      3      7    b  two}

In [14]:
d['a']

,data1,data2,key1,key2
0,1,6,a,one
1,5,9,a,two
4,3,5,a,one


### 按列分组

In [15]:
df.dtypes

data1     int32
data2     int32
key1     object
key2     object
dtype: object

In [16]:
grouped = df.groupby(df.dtypes, axis=1)
dict(list(grouped))

{dtype('int32'):    data1  data2
 0      1      6
 1      5      9
 2      4      7
 3      3      7
 4      3      5, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

### 通过字典进行分组

In [17]:
df = pd.DataFrame(np.random.randint(1, 10, (5, 5)), 
                  columns=['a', 'b', 'c', 'd', 'e'], 
                  index=['Alice', 'Bob', 'Candy', 'Dark', 'Emily'])
df

,a,b,c,d,e
Alice,7,2,7,9,9
Bob,8,7,7,6,8
Candy,4,2,6,6,5
Dark,6,3,9,4,8
Emily,8,4,9,6,5


In [18]:
df.ix[1, 1:3] = np.NaN
df

,a,b,c,d,e
Alice,7,2,7,9,9
Bob,8,NaN,NaN,6,8
Candy,4,2,6,6,5
Dark,6,3,9,4,8
Emily,8,4,9,6,5


In [19]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'orange', 'e': 'blue'}
grouped = df.groupby(mapping, axis=1)

In [20]:
grouped.sum()

,blue,orange,red
Alice,16,9,9
Bob,8,6,8
Candy,11,6,6
Dark,17,4,9
Emily,14,6,12


In [21]:
grouped.count()

,blue,orange,red
Alice,2,1,2
Bob,1,1,1
Candy,2,1,2
Dark,2,1,2
Emily,2,1,2


In [22]:
grouped.size()

blue      2
orange    1
red       2
dtype: int64

### 通过函数来分组

当函数作为分组依据时，数据表里的每个索引（可以是行索引，也可以是列索引）都会调用一次函数，函数的返回值作为分组的索引，即相同的返回值分在同一组。

In [23]:
df = pd.DataFrame(np.random.randint(1, 10, (5, 5)), 
                  columns=['a', 'b', 'c', 'd', 'e'], 
                  index=['Alice', 'Bob', 'Candy', 'Dark', 'Emily'])
df

,a,b,c,d,e
Alice,7,9,1,9,1
Bob,6,6,7,1,5
Candy,7,8,5,3,8
Dark,3,4,6,8,1
Emily,1,2,2,1,2


In [24]:
def _dummy_group(idx):
    print idx
    return idx
df.groupby(_dummy_group)

Alice
Bob
Candy
Dark
Emily


In [25]:
df.groupby(_dummy_group, axis=1)

a
b
c
d
e


In [26]:
grouped = df.groupby(len)

In [27]:
grouped.sum()

,a,b,c,d,e
3,6,6,7,1,5
4,3,4,6,8,1
5,15,19,8,13,11


In [28]:
grouped.size()

3    1
4    1
5    3
dtype: int64

In [29]:
grouped.count()

,a,b,c,d,e
3,1,1,1,1,1
4,1,1,1,1,1
5,3,3,3,3,3


### 多级索引数据根据索引级别来分组

In [30]:
columns = pd.MultiIndex.from_arrays([['China', 'USA', 'China', 'USA', 'China'],
                                     ['A', 'A', 'B', 'C', 'B']], names=['country', 'index'])
df = pd.DataFrame(np.random.randint(1, 10, (5, 5)), columns=columns)
df

country,China,USA,China,USA,China
index,A,A,B,C,B
0,9,6,9,6,2
1,5,6,1,8,7
2,2,5,4,5,2
3,4,8,9,4,9
4,7,2,9,1,8


In [31]:
df.groupby(level='country', axis=1).count()

country,China,USA
0,3,2
1,3,2
2,3,2
3,3,2
4,3,2


In [32]:
df.groupby(level='country', axis=1).sum()

country,China,USA
0,20,12
1,13,14
2,8,10
3,22,12
4,24,3


In [33]:
df.groupby(level='index', axis=1).count()

index,A,B,C
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1
4,2,2,1


## 数据聚合

分组运算，先根据一定规则拆分后的数据，然后对数据进行聚合运算，如前面见到的 `mean()`, `sum()` 等就是聚合的例子。聚合时，拆分后的第一个索引指定的数据都会依次传给聚合函数进行运算。最后再把运算结果合并起来，生成最终结果。

聚合函数除了内置的 `sum()`, `min()`, `max()`, `mean()` 等等之外，还可以自定义聚合函数。自定义聚合函数时，使用 `agg()` 或 `aggregate()` 函数。

### 内置聚合函数

In [34]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randint(1, 10, 5),
                  'data2': np.random.randint(1, 10, 5)})
df

,data1,data2,key1,key2
0,9,3,a,one
1,3,8,a,two
2,9,5,b,one
3,8,5,b,two
4,9,2,a,one


In [35]:
df['data1'].groupby(df['key1']).sum()

key1
a    21
b    17
Name: data1, dtype: int32

### 自定义聚合函数

In [36]:
def peak_verbose(s):
    print type(s)
    return s.max() - s.min()

def peak(s):
    return s.max() - s.min()

In [37]:
grouped = df.groupby('key1')

In [38]:
grouped.agg(peak_verbose)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,data1,data2
key1,,
a,6,6
b,1,0


### 应用多个聚合函数

In [39]:
grouped['data1', 'data2'].agg(['mean', 'std', peak])

data1                    data2              
      mean       std peak      mean      std peak
key1                                             
a      7.0  3.464102    6  4.333333  3.21455    6
b      8.5  0.707107    1  5.000000  0.00000    0

In [40]:
# 给聚合后的列取名
grouped['data1'].agg([('agerage', 'mean'), ('max-range', peak)])

,agerage,max-range
key1,,
a,7.0,6
b,8.5,1


### 给不同的列应用不同的聚合函数

使用 dict 作为参数来实现

In [41]:
d = {'data1': ['mean', peak, 'max', 'min'],
     'data2': 'sum'}
grouped.agg(d)

data1              data2
      mean peak max min   sum
key1                         
a      7.0    6   9   3    13
b      8.5    1   9   8    10

### 重置索引

In [42]:
grouped.agg(d).reset_index()

key1 data1              data2
        mean peak max min   sum
0    a   7.0    6   9   3    13
1    b   8.5    1   9   8    10

In [43]:
df.groupby('key1', as_index=False).agg(d)

key1 data1              data2
        mean peak max min   sum
0    a   7.0    6   9   3    13
1    b   8.5    1   9   8    10

## 分组运算和转换

groupby 是特殊的分组运算。更一般的分组运算包括 “拆分 - 应用 - 合并”。这里介绍 `transform()` 和 `apply()` 来实现分组运算。

### transform

In [44]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randint(1, 10, 5),
                  'data2': np.random.randint(1, 10, 5)})
df

,data1,data2,key1,key2
0,4,9,a,one
1,5,2,a,two
2,1,9,b,one
3,3,9,b,two
4,1,8,a,one


In [45]:
# 给 df 每行都添加一个以 key1 分组后的平均值
k1_mean = df.groupby('key1').mean().add_prefix('mean_')
k1_mean

,mean_data1,mean_data2
key1,,
a,3.333333,6.333333
b,2.000000,9.000000


In [46]:
pd.merge(df, k1_mean, left_on='key1', right_index=True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,4,9,a,one,3.333333,6.333333
1,5,2,a,two,3.333333,6.333333
4,1,8,a,one,3.333333,6.333333
2,1,9,b,one,2.000000,9.000000
3,3,9,b,two,2.000000,9.000000


In [47]:
# 使用 transform 简化处理
k1_mean = df.groupby('key1').transform(np.mean).add_prefix('mean_')
k1_mean

,mean_data1,mean_data2
0,3.333333,6.333333
1,3.333333,6.333333
2,2.000000,9.000000
3,2.000000,9.000000
4,3.333333,6.333333


In [48]:
df[k1_mean.columns] = k1_mean
df

,data1,data2,key1,key2,mean_data1,mean_data2
0,4,9,a,one,3.333333,6.333333
1,5,2,a,two,3.333333,6.333333
2,1,9,b,one,2.000000,9.000000
3,3,9,b,two,2.000000,9.000000
4,1,8,a,one,3.333333,6.333333


### 距平化

与平均值的差异值

In [49]:
df = pd.DataFrame(np.random.randint(1, 10, (5, 5)), 
                  columns=['a', 'b', 'c', 'd', 'e'], 
                  index=['Alice', 'Bob', 'Candy', 'Dark', 'Emily'])
df

,a,b,c,d,e
Alice,4,8,1,7,6
Bob,4,4,4,9,7
Candy,6,2,2,4,6
Dark,4,2,1,4,5
Emily,4,3,4,2,4


In [50]:
def demean(s):
    return s - s.mean()

key = ['one', 'one', 'two', 'one', 'two']
demeaned = df.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Alice,0,3.333333,-1,0.333333,0
Bob,0,-0.666667,2,2.333333,1
Candy,1,-0.500000,-1,1.000000,1
Dark,0,-2.666667,-1,-2.666667,-1
Emily,-1,0.500000,1,-1.000000,-1


In [51]:
demeaned.groupby(key).mean()

,a,b,c,d,e
one,0,-2.960595e-16,0,-2.960595e-16,0
two,0,0.000000e+00,0,0.000000e+00,0


### apply 函数

我们介绍过 DataFrame 的 apply 函数是逐行或逐列来处理数据。GroupBy 的 apply 函数对每个分组进行计算。

In [52]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one', 'one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randint(1, 10, 10),
                  'data2': np.random.randint(1, 10, 10)})
df

,data1,data2,key1,key2
0,3,9,a,one
1,5,9,a,two
2,7,4,b,one
3,7,6,b,two
4,9,7,a,one
5,3,7,a,one
6,3,3,a,two
7,4,5,b,one
8,8,2,b,two
9,7,4,a,one


In [53]:
# 根据 column 排序，输出其最大的 n 行数据
def top(df, n=2, column='data1'):
    return df.sort_values(by=column, ascending=False)[:n]

top(df, n=5)

,data1,data2,key1,key2
4,9,7,a,one
8,8,2,b,two
2,7,4,b,one
3,7,6,b,two
9,7,4,a,one


In [54]:
df.groupby('key1').apply(top)

data1  data2 key1 key2
key1                          
a    4      9      7    a  one
     9      7      4    a  one
b    8      8      2    b  two
     2      7      4    b  one

In [55]:
# 传递参数
df.groupby('key1').apply(top, n=3, column='data2')

data1  data2 key1 key2
key1                          
a    0      3      9    a  one
     1      5      9    a  two
     4      9      7    a  one
b    3      7      6    b  two
     7      4      5    b  one
     2      7      4    b  one

In [56]:
# 禁用分组键
df.groupby('key1', group_keys=False).apply(top)

,data1,data2,key1,key2
4,9,7,a,one
9,7,4,a,one
8,8,2,b,two
2,7,4,b,one


## apply 应用示例：用不同的分组平均值填充空缺数据

In [57]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.479291
New York     -0.301839
Vermont            NaN
Florida       0.476204
Oregon       -0.753084
Nevada             NaN
California   -0.686005
Idaho              NaN
dtype: float64

In [58]:
data.groupby(group_key).mean()

East    0.217885
West   -0.719545
dtype: float64

In [59]:
fill_mean = lambda g: g.fillna(g.mean())

data.groupby(group_key).apply(fill_mean)

Ohio          0.479291
New York     -0.301839
Vermont       0.217885
Florida       0.476204
Oregon       -0.753084
Nevada       -0.719545
California   -0.686005
Idaho        -0.719545
dtype: float64